In [ ]:
import sqlite3
import pandas as pd

df = pd.read_csv("../data/processed/bf_efficiency_clean.csv")

conn = sqlite3.connect("../data/bee_honey.db")
df.to_sql("hive_trials", conn, if_exists="replace", index=False)
conn.close()


In [ ]:
conn = sqlite3.connect("../data/bee_honey.db")

query = """
SELECT hive_id,
       AVG(efficiency) AS avg_efficiency
FROM hive_trials
GROUP BY line_id
ORDER BY avg_efficiency DESC;
"""

pd.read_sql(query, conn)

In [ ]:
query = """
SELECT
    COUNT(*) AS total_trials,
    AVG(blood_input_ml) AS avg_input,
    AVG(egg_output_total) AS avg_output,
    AVG(efficiency) AS avg_efficiency,
    AVG(eggs_per_hour) AS avg_output_rate
FROM hive_trials;
"""

pd.read_sql(query, conn)



In [ ]:
query = """
SELECT
    CAST(pollen_input_g / 500 AS INT) * 500 AS input_bucket,
    COUNT(*) AS n_trials,
    AVG(efficiency) AS avg_efficiency,
    AVG(efficiency * efficiency) - AVG(efficiency) * AVG(efficiency) AS variance
FROM hive_trials
GROUP BY input_bucket
ORDER BY input_bucket;
"""

df_sql = pd.read_sql(query, conn)
df_sql

In [ ]:
conn.close()
